## a) Lectura básica del archivo csv

En primer lugar vamos a hacer la importación en formato CSV, tras ello se hará en formato Pandas que es el que usaremos a partir de ese momento para gestionar los datos a lo largo del documento.

In [193]:
import csv

def cargar_lineas(nombre, n, m):
    
    """
    Función que importa un archivo csv y muestra por pantalla el rango de filas pedidas.
    
    Parameters
    ----------
    nombre : str
        Nombre del archivo csv a arbir.
    n : int
        Fila inicial del rango a mostrar. La primera fila corresponde al índice 0.
    m: int
        Fila final del rango a mostrar.
        
    Return
    ------
    lista[array[str, str, str, int, int]]
        Una lista compuesta por cada una de las líneas pedidas como arrays.
    
    Example
    -------
    >>> cargar_lineas(ine_mortalidad_espanna, 7, 10)
    
    ['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', '2012', '402.950']
    ['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', '2011', '387.911']
    ['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', '2010', '382.047']
    ['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', '2009', '384.933']
    """

    lista_datos = list(csv.reader(open(nombre, "r"), delimiter=";"))
    
    # Tenemos que solucionar dos problemillas, el primero es que el delimitador coma del csv al importar lo entiende
    # como separador de decimales. Este error ocurre en la varaible Total, que por defecto importa como string. 
    # Solucionamos el problema sustituyendo los caracteres delimitadores de decimales (. tras la importación)
    # por nada y luego casteando el tipo a entero de la siguiente manera:
    
    for i in range(1,len(lista_datos)):
        lista_datos[i][-1] = int(lista_datos[i][-1].replace(".", ""))
        
    # El segundo problema es que importa el año como string, pero lo resolvemos facilmente casteandolo como int.

    for i in range(1,len(lista_datos)):
        lista_datos[i][-2] = int(lista_datos[i][-2])
    
    # Podemos impedir la muestra del header en este proceso de importación
    # sumando 1 a los índices del return. El segundo tiene sumado 2 para incluir esta
    # correción y aparte incluir el último término que le pasamos como argumento m.
    
    return(lista_datos[n+1:m+2])

Probamos la función elaborada cargando las filas desde la 7 hasta las 10, las dos incluidas.

In [194]:
mis_datos = cargar_lineas("ine_mortalidad_espanna.csv", 7, 10)

for dato in mis_datos:
    print(dato)

['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', 2011, 387911]
['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', 2010, 382047]
['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', 2009, 384933]
['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', 2008, 386324]


Probamos que los tipos están correctos en las dos varables numéricas probando una de las líneas:

In [197]:
for i in range(5):
    print(mis_datos[3][i], type(mis_datos[3][i]))


001-102  I-XXII.Todas las causas <class 'str'>
Total <class 'str'>
Todas las edades <class 'str'>
2008 <class 'int'>
386324 <class 'int'>


Podemos realizar el mismo proceso utilizando los DF de Pandas que será el formato que utilizaremos de ahora en adelante en el proyecto.

In [187]:
import pandas as pd

def cargar_lineas_pd(nombre, n, m):
    
    """
    Función que importa un archivo csv y muestra por pantalla el rango de filas pedidas.
    
    Parameters
    ----------
    nombre : str
        Nombre del archivo csv a arbir.
    n : int
        Fila inicial del rango a mostrar. La primera fila corresponde al índice 0.
    m: int
        Fila final del rango a mostrar.
        
    Return
    ------
    pandas.core.frame.DataFrame (Colección de filas de Pandas DataFrame.)
    
    
    """

    # En este caso, la importación de la columna Periodo se hace como entero automaticamente, pero debido a los
    # delimitadores decimales del csv original, en la columna total nos hace conversiones donde interpreta que
    # la coma, en lugar de separar millares es un punto que separa decimales. Mi manera de solucionarlo es la siguiente.
    
    # Importamos el csv declarando la columna Total como str para que no haga cambios extraños por su cuenta.
    
    DF = pd.read_csv(nombre, delimiter = ";",dtype = {"Total": str})
    
    # Ahora estamos en la situación del apartado anterior y podemos tomar medidas equivalentes.
    # Podemos actuar sobre todos los valores de determinadas columnas.
    # Reemplazamos las apariciones de "." por nada utilizando expresiones regulares.
    
    DF["Total"] = DF["Total"].replace(regex=["\."],value="")
    
    # Tras ello, casteamos la columna objetivo como enteros y el resto como sus tipos deseados.
    
    DF = DF.astype({"Causa de muerte":"string", "Sexo":"string", "Edad":"string", "Periodo":"int32","Total": "int64"})
    
        
    return(DF[n:m+1])

In [188]:
datos = cargar_lineas_pd("ine_mortalidad_espanna.csv", 7, 10)

Podemos ver el tipo de objeto devuelto por la función y comprobar que es un DF:

In [192]:
type(datos)

pandas.core.frame.DataFrame

Hacemos una preview de las filas deseadas:

In [189]:
datos

,Causa de muerte,Sexo,Edad,Periodo,Total
7,001-102 I-XXII.Todas las causas,Total,Todas las edades,2011,387911
8,001-102 I-XXII.Todas las causas,Total,Todas las edades,2010,382047
9,001-102 I-XXII.Todas las causas,Total,Todas las edades,2009,384933
10,001-102 I-XXII.Todas las causas,Total,Todas las edades,2008,386324


Y comprobamos tipos:

In [190]:
datos.dtypes

Causa de muerte    string
Sexo               string
Edad               string
Periodo             int32
Total               int64
dtype: object

Podemos mostrar por pantalla los datos devueltos por la función de csv.reader para ver que a los mismos argumentos les corresponden la misma salida:

In [198]:
mis_datos = cargar_lineas("ine_mortalidad_espanna.csv", 7, 10)

for dato in mis_datos:
    print(dato)

['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', 2011, 387911]
['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', 2010, 382047]
['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', 2009, 384933]
['001-102  I-XXII.Todas las causas', 'Total', 'Todas las edades', 2008, 386324]


# Líneas de código de pruebas

In [155]:
#DF = pd.read_csv("ine_mortalidad_espanna.csv", delimiter = ";",dtype = {'Total': str})

In [141]:
#type(DF.iloc[3,-1])

str

In [156]:
#DF["Total"] = DF["Total"].replace(regex=["\."],value="")
#DF["Total"]

0         427721
1         424523
2         410611
3         422568
4         395830
           ...  
301153         0
301154         0
301155         0
301156         0
301157         0
Name: Total, Length: 301158, dtype: object

In [158]:
#DF2 = DF.astype({'Total': 'int32'}).dtypes

In [160]:
#DF2["Total"]

dtype('int32')

In [97]:
#DF.iloc[3,-1]

'422.568'

In [82]:
#type((DF.iloc[50,-1]))

int